In [58]:
import csv
import gzip
import os
import scipy.io
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE

In [44]:
pwd

'/Users/lidiayung/github/notebooks/GBM'

In [56]:
# define MEX directory
matrix_dir = "/Users/lidiayung/project/resource/GSM5319572_SF4324/"
# read in MEX format matrix as table
mat = scipy.io.mmread(os.path.join(matrix_dir, "matrix.mtx.gz"))
 
# list of transcript ids, e.g. 'ENSG00000243485'
features_path = os.path.join(matrix_dir, "features.tsv.gz")
#feature_ids = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of gene names, e.g. 'MIR1302-2HG'
gene_names = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of feature_types, e.g. 'Gene Expression'
feature_types = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv.gz")
barcodes = [row[0] for row in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]

In [4]:
import pandas as pd
 
# transform table to pandas dataframe and label rows and columns
matrix = pd.DataFrame.sparse.from_spmatrix(mat)
matrix.columns = barcodes
#matrix.insert(loc=0, column="feature_id", value=feature_ids)
#matrix.insert(loc=0, column="feature_type", value=feature_types)
matrix=matrix.transpose() 
# display matrix
matrix.columns = gene_names
print(matrix.head())
# save the table as a CSV (note the CSV will be a very large file)
#matrix.to_csv("mex_matrix.csv", index=False)

                    RP11-34P13.3  FAM138A  OR4F5  RP11-34P13.7  RP11-34P13.8  \
AAACCCAAGAAGGTAG-1             0        0      0             0             0   
AAACCCACAGAGTGAC-1             0        0      0             0             0   
AAACCCAGTCATGGCC-1             0        0      0             0             0   
AAACCCATCGTTACCC-1             0        0      0             0             0   
AAACCCATCTCCACTG-1             0        0      0             0             0   

                    RP11-34P13.14  RP11-34P13.9  FO538757.3  FO538757.2  \
AAACCCAAGAAGGTAG-1              0             0           0           1   
AAACCCACAGAGTGAC-1              0             0           0           1   
AAACCCAGTCATGGCC-1              0             0           0           1   
AAACCCATCGTTACCC-1              0             0           0           0   
AAACCCATCTCCACTG-1              0             0           0           0   

                    AP006222.2  ...  AC007325.2  BX072566.1  AL35482

In [5]:
matrix.shape

(3688, 33694)

In [6]:
newlist =[]
for x in barcodes:
    newlist.append(x.replace('-1', ''))


In [7]:
matrix["Barcode"]=newlist

In [8]:
file = "/Users/lidiayung/project/resource/GSM5319572_SF4324/GSE174554_Tumor_normal_metadata.txt"

In [9]:
metadata= pd.read_csv(file,sep=' ')
metadata.head()

,Sample#,Barcode,Tumor_Normal_annotation
0,SF10022,CTATCTAAGCAAGCCA,Tumor
1,SF10022,AAACCCAGTCTACGAT,Normal
2,SF10022,AAAGGGCTCACCCTGT,Normal
3,SF10022,AACAACCAGACCCGCT,Normal
4,SF10022,AACAAGAGTGTAAACA,Normal


In [10]:
metadata.groupby("Sample#")["Tumor_Normal_annotation"].count()

Sample#
SF10022      3082
SF10099       568
SF10099v2     234
SF10108      4446
SF10127      1052
             ... 
SF9715v2     3731
SF9791v2     1608
SF9798       4051
SF9871       3602
SF9962       3468
Name: Tumor_Normal_annotation, Length: 100, dtype: int64

In [11]:
sum = metadata.groupby("Sample#")["Tumor_Normal_annotation"].count()

In [12]:
pd.DataFrame(sum)

,Tumor_Normal_annotation
Sample#,
SF10022,3082
SF10099,568
SF10099v2,234
SF10108,4446
SF10127,1052
...,...
SF9715v2,3731
SF9791v2,1608
SF9798,4051


tumor_count=[]
for i in metadata["Sample#"]:
    tumor_count = metadata[[[metadata["Sample#"]== i]|metadata["Tumor_Normal_annotation"]=="Tumor"]]
    tumor_count.append(tumor_count)

In [13]:
table= pd.pivot_table(metadata,index=['Sample#','Tumor_Normal_annotation'],aggfunc='count')

In [14]:
tumor_total = metadata[metadata['Tumor_Normal_annotation'] == 'Tumor']

In [15]:
tumor_total

,Sample#,Barcode,Tumor_Normal_annotation
0,SF10022,CTATCTAAGCAAGCCA,Tumor
172,SF10022,TCACTATAGATGCGAC,Tumor
185,SF10022,AGGACGAGTGTGCTTA,Tumor
186,SF10022,ATAGAGATCCCTGTTG,Tumor
187,SF10022,CCAATTTAGGCTTTCA,Tumor
...,...,...,...
254282,SF9791v2,TTTATGCTCTCCTGAC,Tumor
254283,SF9791v2,TTTCAGTCATCTTCGC,Tumor
254284,SF9791v2,TTTCGATAGTCATAGA,Tumor
254285,SF9791v2,TTTGGAGCACTGTCGG,Tumor


tumor_percentage=[]
for i in metadata['Sample#']:
    tumor_total = metadata[metadata['Tumor_Normal_annotation'] == 'Tumor']
    tumor_count = tumor_total[tumor_total["Sample#"]== i].count()
    total=metadata[metadata["Sample#"]== i].count()
    percentage = tumor_count/total*100
    tumor_percentage.append(percentage)

In [16]:
tumor_count = tumor_total[tumor_total["Sample#"]== "SF9791v2"].count()
print(tumor_count)

Sample#                    1339
Barcode                    1339
Tumor_Normal_annotation    1339
dtype: int64


In [17]:
table.describe

<bound method NDFrame.describe of                                    Barcode
Sample#   Tumor_Normal_annotation         
SF10022   Normal                      2490
          Tumor                        592
SF10099   Normal                       119
          Tumor                        449
SF10099v2 Normal                       232
...                                    ...
SF9798    Tumor                       3850
SF9871    Normal                      3596
          Tumor                          6
SF9962    Normal                       808
          Tumor                       2660

[200 rows x 1 columns]>

percentage=[]
for i,h in table['count']:
    percentage = h/(i+h)*100
    percentage.append(percentage)
print(percentage)

In [18]:
#table.to_csv("tumor_percentage_table.csv",sep=',',index=True)

In [37]:
SF1199 = metadata[metadata['Sample#']=='SF1199']
SF1199 = SF1199[SF1199['Tumor_Normal_annotation']=="Tumor"]

SF9962.shape

(2660, 3)

In [38]:
SF1199

,Sample#,Barcode,Tumor_Normal_annotation
75177,SF1199,AAGCGTTGTATAGCTC,Tumor
75179,SF1199,CGTTCTGCACGTAACT,Tumor
75180,SF1199,GGATCTAAGCTAGAGC,Tumor
75182,SF1199,TAAGCCACATAGCTGT,Tumor
75183,SF1199,TCATCCGCAATTGCCA,Tumor
...,...,...,...
75806,SF1199,GCCCGAAAGGCTCTAT,Tumor
75807,SF1199,TCAAGACCAAACTAGA,Tumor
75808,SF1199,TCATTGTGTCGCACGT,Tumor
75809,SF1199,TTAATCCAGTCACTGT,Tumor


In [33]:
SF9962 = metadata[metadata['Sample#']=='SF9962']
SF9962= SF9962[SF9962['Tumor_Normal_annotation']=="Tumor"]

In [34]:
SF9962.shape

(2660, 3)

In [39]:
SF9962

,Sample#,Barcode,Tumor_Normal_annotation
181164,SF9962,CAAGGGACAAGCTCTA,Tumor
181179,SF9962,GCACGTGAGGAAGTCC,Tumor
181180,SF9962,GCATTAGGTGCATACT,Tumor
181185,SF9962,TAAGCGTCAACATCGT,Tumor
181187,SF9962,TAGGTTGCAAACGGCA,Tumor
...,...,...,...
184614,SF9962,TTCGGTCAGTTGAAGT,Tumor
184615,SF9962,TTTACTGCATGGACAG,Tumor
184617,SF9962,GACCCTTGTTGAGGAC,Tumor
184618,SF9962,GAACTGTAGCCTCACG,Tumor


In [21]:
merge = matrix.merge(SF9962, on="Barcode")
data = matrix[matrix['Barcode'].isin(merge["Barcode"])]
data.shape

(893, 33695)

In [22]:
data.head()

,RP11-34P13.3,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,RP11-34P13.14,RP11-34P13.9,FO538757.3,FO538757.2,AP006222.2,...,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.1,FAM231B,Barcode
AAACGAAAGTGTTGTC-1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,AAACGAAAGTGTTGTC
AAACGAATCGTAACTG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AAACGAATCGTAACTG
AAACGCTCAAATAGCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AAACGCTCAAATAGCA
AAAGGATGTCGTTGGC-1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,AAAGGATGTCGTTGGC
AAAGGGCTCTTCACGC-1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,AAAGGGCTCTTCACGC


In [72]:
data.drop(columns='Barcode')

,RP11-34P13.3,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,RP11-34P13.14,RP11-34P13.9,FO538757.3,FO538757.2,AP006222.2,...,AC007325.2,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.1,FAM231B
AAACGAAAGTGTTGTC-1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
AAACGAATCGTAACTG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACGCTCAAATAGCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAGGATGTCGTTGGC-1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
AAAGGGCTCTTCACGC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCAGTCATACTTTC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
TTTCATGAGCTGTGCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTCATGGTACCGGAA-1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
TTTGACTAGTATGTAG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
data['Barcode']

AAACGAAAGTGTTGTC-1    AAACGAAAGTGTTGTC
AAACGAATCGTAACTG-1    AAACGAATCGTAACTG
AAACGCTCAAATAGCA-1    AAACGCTCAAATAGCA
AAAGGATGTCGTTGGC-1    AAAGGATGTCGTTGGC
AAAGGGCTCTTCACGC-1    AAAGGGCTCTTCACGC
                            ...       
TTTCAGTCATACTTTC-1    TTTCAGTCATACTTTC
TTTCATGAGCTGTGCC-1    TTTCATGAGCTGTGCC
TTTCATGGTACCGGAA-1    TTTCATGGTACCGGAA
TTTGACTAGTATGTAG-1    TTTGACTAGTATGTAG
TTTGTTGCAAGTTGGG-1    TTTGTTGCAAGTTGGG
Name: Barcode, Length: 893, dtype: object

In [25]:
data.to_csv("SF4324_tumor_matrix.csv", index=True)

In [ ]:

# define MEX directory
mtrix_dir = "/Users/lidiayung/project/resource/GSM5319523_SF4209/"
# read in MEX format matrix as table

mat = scanpy.read_10x_mtx(os.path.join(mtrix_dir, "matrix.mtx.gz"))
 
# list of transcript ids, e.g. 'ENSG00000243485'
features_path = os.path.join(matrix_dir, "features.tsv.gz")
#feature_ids = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of gene names, e.g. 'MIR1302-2HG'
gene_names = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
 
# list of feature_types, e.g. 'Gene Expression'
feature_types = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv.gz")
barcodes = [row[0] for row in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]

In [51]:
f_exprMat = '/Users/lidiayung/github/notebooks/GBM/SF4324_tumor_matrix.csv'

adata = sc.read_csv( f_exprMat, delimiter=',', first_column_names=True, dtype='str' )

In [52]:
adata

AnnData object with n_obs × n_vars = 893 × 33695